<a href="https://colab.research.google.com/github/lhccd/mini_projects/blob/main/CSS_SemanticDrift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Retrieval + Transformation

Interesting Findings:
- training corpus has a size of 3 GB, which is still small – for your model, you will get better results the more data you can get to pretrain on (https://huggingface.co/blog/how-to-train)
- We tokenize those English Tweets using “TweetTokenizer” from the NLTK toolkit (Bird et al., 2009) and use the emoji package to translate emotion icons into text strings (here, each icon isreferredtoasawordtoken).**bold text** We also normalize the Tweets by converting user mentions and web/url links into special tokens @USER and HTTPURL, respectively. We filter out retweeted Tweets and the ones shorter than 10 or longer than 64 word tokens. This pre-process results in the first corpus of 845M English Tweets. (http://aclanthology.lst.uni-saarland.de/2020.emnlp-demos.2.pdf)


In [ ]:
# read file

import json

with open('/content/ukraine_20220613.json', 'r') as f:
  data = json.load(f)




# Neuer Abschnitt

In [ ]:
# concatenate tweets to one string 
## TODO: check if one should indicate the end/start of a tweet

txt = ""

for tweet in data: 
  txt = txt + tweet["text"]

NameError: ignored

In [ ]:
# perform data cleaning
## TODO: remove links/ user mentions

# lower case txt
txt = txt.lower()

# remove emojis
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

txt = deEmojify(txt)


In [ ]:
# save text file

#open text file
text_file = open('/content/train.txt', 'w')
 
#write string to file
text_file.write(txt)
 
#close file
text_file.close()

#Train Tokenizer

In [ ]:
!pip install transformers
!pip install tokenizers

from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.20.0-py3-none-any.whl (4.4 MB)
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


ModuleNotFoundError: ignored

In [ ]:
paths = ['/content/train.txt']

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

!mkdir Putin_ML
tokenizer.save_model("Putin_ML")


In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./Putin_ML/vocab.json",
    "./Putin_ML/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [1]:
tokenizer.encode("This is putin").tokens

NameError: ignored

#Train Language Model

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./Putin_ML", max_len=512)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./train.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./Putin_ML",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:
trainer.train()
trainer.save_model("./Putin_ML")

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7185
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 113
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=113, training_loss=8.80671138257052, metrics={'train_runtime': 130.4463, 'train_samples_per_second': 55.08, 'train_steps_per_second': 0.866, 'total_flos': 238231493959680.0, 'train_loss': 8.80671138257052, 'epoch': 1.0})

Test Language Model

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./Putin_ML",
    tokenizer="./Putin_ML"
)

loading configuration file ./Putin_ML/config.json
Model config RobertaConfig {
  "_name_or_path": "./Putin_ML",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./Putin_ML/config.json
Model config RobertaConfig {
  "_name_or_path": "./Putin_ML",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,


In [ ]:
fill_mask("Putin loves <mask>.")

[{'score': 0.10590218007564545,
  'sequence': 'Putin loves..',
  'token': 18,
  'token_str': '.'},
 {'score': 0.009909302927553654,
  'sequence': 'Putin loves the.',
  'token': 269,
  'token_str': ' the'},
 {'score': 0.006747542880475521,
  'sequence': 'Putin loves #.',
  'token': 309,
  'token_str': ' #'},
 {'score': 0.006165325175970793,
  'sequence': 'Putin loves,.',
  'token': 16,
  'token_str': ','},
 {'score': 0.005702795460820198,
  'sequence': 'Putin loves://.',
  'token': 316,
  'token_str': '://'}]

In [ ]:
model(torch.tensor(tokenizer.encode("putin")).unsqueeze(0).to(device='cuda'))

AttributeError: ignored